In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
"""Regeneration of lead synth from combined signal"""
import keras
from keras.layers import Dense, Dropout, PReLU
from keras.optimizers import Adam

from music_generator.basic.random import generate_dataset
from music_generator.basic.signalproc import SamplingInfo
from music_generator.musical.timing import Tempo
from music_generator.musical.scales import GenericScale
from music_generator.analysis.play import play_mono_as_stereo, play_array
from music_generator.basic.signalproc import mix_at
from music_generator.analysis import preprocessing

from music_generator.musical import scales
import numpy as np
from multiprocessing import Pool
from functools import partial

import matplotlib.pyplot as plt
from IPython.display import Audio
from scipy.io.wavfile import read
from music_generator.analysis import regen_models
from scipy.io import wavfile
from music_generator.analysis import jamdataset
import tensorflow as tf

from music_generator.analysis import regen_models
import importlib

In [ ]:
sr = 44100
sampling_info = SamplingInfo(sr)
n_samples = 4096
fragment_length = 4096

In [ ]:
model = regen_models.FftBranchesFilter(n_samples, learning_rate=1e-4, loss_fct='fft_loss').build_model()
model.summary()

In [ ]:
while True:

    jds = jamdataset.JamDataSet(fragment_length=fragment_length, n_samples_train=10000, n_samples_test=500)
    model.fit(jds.x_train, 
              jds.y_train, 
              epochs=5, 
              verbose=1,
              shuffle=True,
              batch_size=64,
              validation_data=[jds.x_test, jds.y_test])
    
    pred_model = regen_models.RegenModel(model=model, fragment_length=fragment_length)

    sr, dt = wavfile.read('../data/full-mix.wav')
    dt = dt / 2**16

    pred = pred_model.predict(dt)
    play_array(pred[20*sr:40*sr], do_wait_done=False)

In [ ]:
# Audio(pred, rate=sr)

In [ ]:
# start_sec, end_sec = 20, 60

# pred = model_predict(model, input_track[start_sec*sr:end_sec*sr], fragment_length)
# play_array(input_track[start_sec*sr:end_sec*sr])
# play_array(pred)

# #%%

# x_pred = input_track[start_sec*sr:end_sec*sr]
# x_pred = x_pred[:(len(x_pred) // fragment_length) * fragment_length]
# x_pred = x_pred.reshape(-1, fragment_length)

# y_pred = model.predict(x_pred)
# play_array(y_pred)